# 作業 : (Kaggle)鐵達尼生存預測

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成自己挑選特徵的群聚編碼 (In[2], Out[2])
- 觀察群聚編碼, 搭配邏輯斯回歸, 看看有什麼影響 (In[5], Out[5], In[6], Out[6]) 

# 作業1
* 試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

In [1]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

data_path = '../data/'
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [13]:
df['SibSp'].unique()

array([1, 0, 3, 4, 2, 5, 8])

In [14]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼
"""
Your Code Here
"""
# mean_df = df.groupby(['SaleCondition'])['GrLivArea'].mean().reset_index()
# mode_df = df.groupby(['SaleCondition'])['GrLivArea'].apply(lambda x: x.mode()[0]).reset_index()
# median_df = df.groupby(['SaleCondition'])['GrLivArea'].median().reset_index()
# max_df = df.groupby(['SaleCondition'])['GrLivArea'].max().reset_index()
# temp = pd.merge(mean_df, mode_df, how='left', on=['SaleCondition'])
# temp = pd.merge(temp, median_df, how='left', on=['SaleCondition'])
# temp = pd.merge(temp, max_df, how='left', on=['SaleCondition'])
# temp.columns = ['SaleCondition', 'Area_Sale_Mean', 'Area_Sale_Mode', 'Area_Sale_Median', 'Area_Sale_Max']
# temp
# df['SaleCondition'] = df['SaleCondition'].fillna('None')


df['SibSp'] = df['SibSp'].fillna('None')
mean_df = df.groupby(['SibSp']) ['Age'].mean().reset_index()
mode_df = df.groupby(['SibSp'])['Age'].apply(lambda x: x.mode()).reset_index()
median_df = df.groupby(['SibSp'])['Age'].median().reset_index()
max_df = df.groupby(['SibSp'])['Age'].max().reset_index()
temp = pd.merge(mean_df, mode_df, how='left', on=['SibSp'])
temp = pd.merge(temp, median_df, how='left', on=['SibSp'])
temp = pd.merge(temp, max_df, how='left', on=['SibSp'])
temp

,SibSp,Age_x,level_1,Age_y,Age_x,Age_y
0,0,31.101954,0.0,22.0,29.0,80.0
1,1,30.173212,0.0,25.0,31.0,70.0
2,2,20.797619,0.0,24.0,21.0,53.0
3,3,11.384615,0.0,2.0,9.0,33.0
4,4,6.794118,0.0,2.0,7.0,17.0
5,5,10.200000,0.0,1.0,11.0,16.0
6,5,10.200000,1.0,9.0,11.0,16.0
7,5,10.200000,2.0,11.0,11.0,16.0
8,5,10.200000,3.0,14.0,11.0,16.0
9,5,10.200000,4.0,16.0,11.0,16.0


In [15]:
df = pd.merge(df, temp, how='left', on=['SibSp'])
df = df.drop(['SibSp'] , axis=1)
df.head()

,Pclass,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Age_x_x,level_1_x,Age_y_x,Age_x_x,Age_y_x,Age_x_y,level_1_y,Age_y_y,Age_x_y,Age_y_y
0,3,"Braund, Mr. Owen Harris",male,22.0,0,7.2500,NaN,S,22.0,0.0,22.0,22.0,22.0,30.173212,0.0,25.0,31.0,70.0
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,0,71.2833,C85,C,38.0,0.0,38.0,38.0,38.0,30.173212,0.0,25.0,31.0,70.0
2,3,"Heikkinen, Miss. Laina",female,26.0,0,7.9250,NaN,S,26.0,0.0,26.0,26.0,26.0,31.101954,0.0,22.0,29.0,80.0
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,0,53.1000,C123,S,36.0,0.0,35.0,36.0,37.0,30.173212,0.0,25.0,31.0,70.0
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,0,53.1000,C123,S,36.0,1.0,37.0,36.0,37.0,30.173212,0.0,25.0,31.0,70.0


In [17]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
df.head()

22 Numeric Features : ['Pclass', 'Age', 'Parch', 'Fare', 'Age_x_x', 'Age_x_x', 'level_1_x', 'Age_y_x', 'Age_y_x', 'Age_x_x', 'Age_x_x', 'Age_y_x', 'Age_y_x', 'Age_x_y', 'Age_x_y', 'level_1_y', 'Age_y_y', 'Age_y_y', 'Age_x_y', 'Age_x_y', 'Age_y_y', 'Age_y_y']



,Pclass,Age,Parch,Fare,Age_x_x,Age_x_x,Age_x_x,Age_x_x,Age_x_x,Age_x_x,...,Age_x_y,Age_x_y,Age_y_y,Age_y_y,Age_y_y,Age_y_y,Age_y_y,Age_y_y,Age_y_y,Age_y_y
0,3,22.0,0,7.2500,22.0,22.0,22.0,22.0,22.0,22.0,...,30.173212,31.0,25.0,70.0,25.0,70.0,25.0,70.0,25.0,70.0
1,1,38.0,0,71.2833,38.0,38.0,38.0,38.0,38.0,38.0,...,30.173212,31.0,25.0,70.0,25.0,70.0,25.0,70.0,25.0,70.0
2,3,26.0,0,7.9250,26.0,26.0,26.0,26.0,26.0,26.0,...,31.101954,29.0,22.0,80.0,22.0,80.0,22.0,80.0,22.0,80.0
3,1,35.0,0,53.1000,36.0,36.0,36.0,36.0,36.0,36.0,...,30.173212,31.0,25.0,70.0,25.0,70.0,25.0,70.0,25.0,70.0
4,1,35.0,0,53.1000,36.0,36.0,36.0,36.0,36.0,36.0,...,30.173212,31.0,25.0,70.0,25.0,70.0,25.0,70.0,25.0,70.0


# 作業2
* 將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?

In [ ]:
# 原始特徵 + 邏輯斯迴歸
"""
Your Code Here
"""

In [ ]:
# 新特徵 + 邏輯斯迴歸
"""
Your Code Here
"""